# Solar Flare prediction dataset pipeline

This is a replication of the dataset used by [Liu et al.](https://web.njit.edu/~wangj/LSTMpredict/).

The data comes from 2 sources:

1. Flare data from the GOES flare catalog at NOAA, which can be accessed with the sunpy.instr.goes.get_event_list() function.
 This tells us if an active region produced a flare or not.
2. Active region data from the Solar Dynamics Observatory's Heliosesmic and Magnetic Imager instrument, which can be accessed from the JSOC database via a JSON API.
This gives us the features characterizing each active region.

We ascribe each Active Region (AR) to one of two classes:

1. The positive class contains flaring active regions that will produce
flare >M5.0 in the next 24hours.
2. The negative class contains flaring active regions that will **not**
produce flare >M5.0 in the next 24hours.

First, some imports.


In [77]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.mlab as mlab
import pandas as pd
import scipy.stats
import requests
import urllib
import json
from datetime import datetime as dt_obj
from datetime import timedelta
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sunpy.time import TimeRange
from sunpy.net import hek
from astropy.time import Time
import sunpy.instr.goes
import lime
import lime.lime_tabular
import os
import drms
pd.set_option('display.max_rows', 100)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Step 1: Get flare list

We get the entire GOES flare catalog at NOAA.



In [78]:
# Grab all the data from the GOES database
t_start = "2010-05-01"
t_end = "2018-05-11"
time_range = TimeRange(t_start, t_end)
if os.path.exists("../Data/GOES/all_flares_list.csv"):
    listofresults = pd.read_csv('../Data/GOES/all_flares_list.csv').drop\
        (columns="Unnamed: 0")
else:
    listofresults = sunpy.instr.goes.get_goes_event_list(time_range, 'B1')
    # Remove all events without NOAA number
    listofresults = listofresults[listofresults['noaa_active_region'] != 0]
    # save to csv
    pd.DataFrame(listofresults).to_csv('../Data/GOES/all_flares_list.csv')
    listofresults = pd.DataFrame(listofresults)

listofresults = listofresults.sort_values(by=['noaa_active_region',
                                            'start_time']).reset_index(drop=True)
print('Grabbed all the GOES data; there are', len(listofresults), 'events.')

Grabbed all the GOES data; there are 11986 events.


Convert the ```times``` in the ```listofresults``` dataframe from a string
into a datetime object:

In [79]:
def parse_tai_string(tstr):
    year = int(tstr[:4])
    month = int(tstr[5:7])
    day = int(tstr[8:10])
    hour = int(tstr[11:13])
    minute = int(tstr[14:16])
    return dt_obj(year, month, day, hour, minute)

listofresults['start_time'] = listofresults['start_time'].apply(parse_tai_string)
listofresults['peak_time'] = listofresults['peak_time'].apply(parse_tai_string)
listofresults['end_time'] = listofresults['end_time'].apply(parse_tai_string)

Now let's query the JSOC database to see if there are active region parameters at the time of the flare.
First read the following file to map NOAA active region numbers to HARPNUMs (a HARP, or an HMI Active Region Patch, is the preferred numbering system for the HMI active regions as they appear in the magnetic field data before NOAA observes them in white light):

In [80]:
HARP_NOAA_list = pd.read_csv(
    'http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt', sep=' ')

Now, let's determine at which time we'd like to predict CMEs. In general,
many people try to predict a flare either 24 or 48 hours before it happens.
We can report both in this study by setting a variable called ```timedelayvariable```:

In [81]:
timedelayvariable = 24

# Step 2: Get SHARP data

Now we can grab the SDO data from the JSOC database by executing the JSON queries.
We are selecting data that satisfies several criteria:
The data has to be [1] disambiguated with a version of the disambiguation module greater than 1.1,
 [2] taken while the orbital velocity of the spacecraft is less than 3500 m/s,
 [3] of a high quality, and
 [4] within 70 degrees of central meridian.
 If the data pass all these tests, they are stuffed into one dataframe
 ```data_jsoc```

We prepare the data tobe fed into function, with all the >M5.0 class flares
labelled positive the ```timedelayvariable``` ahead of time.


In [82]:
minimum_class_label = ['M5', 'M6', 'M7', 'M8', 'M9', 'X']
listofactiveregions = list(listofresults['noaa_active_region'].unique())
FL_GOESCLS_MAJOR_DICT = {'B': 1.0, 'C' : 2.0, 'M': 3.0, 'X': 4.0}

In [83]:
def get_the_jsoc_data(event_count):
    """
    Parameters
    ----------
    event_count: number of events
                 int

    t_rec:       list of times, one associated with each event in event_count
                 list of strings in JSOC format ('%Y.%m.%d_%H:%M_TAI')

    """
    from astropy.time import Time
    start_date = drms.to_datetime(t_start).strftime('%Y.%m.%d_%H:%M_TAI')
    end_date = drms.to_datetime(t_end).strftime('%Y.%m.%d_%H:%M_TAI')
    series_sharp = 'hmi.sharp_cea_720s'
    series_lorentz = 'cgem.lorentz'
    ids = ['T_REC','NOAA_AR', 'HARPNUM', 'CRVAL1','CRVAL2', 'CRLN_OBS',
           'CRLT_OBS', 'LAT_FWT', 'LON_FWT']
    sharps = ['USFLUX', 'MEANGBT',
              'MEANJZH', 'MEANPOT', 'SHRGT45',
              'TOTUSJH', 'MEANGBH','MEANALP','MEANGAM','MEANGBZ','MEANJZD',
              'TOTUSJZ','SAVNCPP', 'TOTPOT','MEANSHR','AREA_ACR','R_VALUE',
              'ABSNJZH']
    lorentzs = ['TOTBSQ', 'TOTFX','TOTFY','TOTFZ','EPSX','EPSY','EPSZ']
    conditions = '(CODEVER7 !~ "1.1") and (abs(OBS_VR)< 3500) and (QUALITY<65536)'
    conditions_lor = '(abs(OBS_VR)< 3500) and (QUALITY<65536)'
    c = drms.Client()
    data_jsoc = pd.DataFrame()

    # for earch active region
    for i in range(event_count):

        print("=====", i, "=====")
        # next match NOAA_ARS to HARPNUM
        idx = HARP_NOAA_list[HARP_NOAA_list['NOAA_ARS'].str.contains(
            str(int(listofactiveregions[i])))]

        # if there's no HARPNUM, quit
        if (idx.empty == True):
            print('skip: there are no matching HARPNUMs for',
                  str(int(listofactiveregions[i])))
            continue

        harpnum = idx.HARPNUM.values[0]
        # query jsoc database for sharp data
        data_sharp = c.query('%s[%d][%s-%s@60m][? %s ?]' % (series_sharp,
                                                           harpnum,
                                                        start_date,
                                                        end_date,
                                                   conditions),
                       key=ids+sharps)

        # if there are no data at this time, quit
        if len(data_sharp) == 0:
            print('skip: there are no data for HARPNUM',
                  harpnum)
            continue

        # query jsoc database for lorentz data
        data_lorentz = c.query('%s[%d][%s-%s@60m][? %s ?]' % (series_lorentz,harpnum,
                                                        start_date,
                                                        end_date,
                                                   conditions_lor),
                       key=lorentzs)

                # if there are no data at this time, quit
        if len(data_lorentz) == 0:
            print('skip: there are no data for HARPNUM',
                  harpnum)
            continue

        #concat the tables
        data = pd.concat([data_sharp, data_lorentz], axis=1)

        # check to see if the active region is too close to the limb
        # we can compute the latitude of an active region in stonyhurst coordinates as follows:
        # longitude_stonyhurst = CRVAL1 - CRLN_OBS
        # for this we have to query the CEA series (but above we queried the other series as the CEA series does not have CODEVER5 in it)
        data = data[np.abs(data['LON_FWT']) < 70.0]

        # convert tai string to date time
        data['T_REC'] = data['T_REC'].apply(parse_tai_string)

        print('accept NOAA Active Region number', str(int(
            listofactiveregions[i])), 'and HARPNUM', harpnum)

        # Append to larger dataset
        data_jsoc = pd.concat([data_jsoc, data], ignore_index=True)
        # append to csv
        outfile = '../Data/SHARP/jsoc_data.csv'
        data.to_csv(outfile, mode='a', header=not os.path.exists(outfile),
                    index=False)

    return data_jsoc

Call the function

In [84]:
if os.path.exists('../Data/SHARP/jsoc_data.csv'):
    data_jsoc = pd.read_csv('../Data/SHARP/jsoc_data.csv')
else:
    data_jsoc = get_the_jsoc_data(len(listofactiveregions))
# data_jsoc = get_the_jsoc_data(len(listofactiveregions))

## Match data with flares

Flare to flux function for determining the Xmax1d data.

In [85]:
def flare_to_flux(flare_class):
    flux = 1e-7
    if flare_class == 'N':
        flux = 1e-7
    else:
        class_label = flare_class[0]
        class_mag = float(flare_class[1:])
        if class_label=='B':
            flux = 1e-7 * class_mag
        if class_label=='C':
            flux = 1e-6 * class_mag
        if class_label=='M':
            flux = 1e-5 * class_mag
        if class_label=='X':
            flux = 1e-4 * class_mag
    return flux


In [86]:
# extra cleanup
data_jsoc = data_jsoc.drop(columns=['CRVAL1', 'CRVAL2', 'CRLN_OBS',
                                    'CRLT_OBS'])
data_jsoc['T_REC'] = pd.to_datetime(data_jsoc['T_REC'])
data_jsoc = data_jsoc.sort_values(by=['HARPNUM', 'T_REC']).reset_index\
    (drop=True)
# prepare columns
data_jsoc.insert(0, 'flare', 'N')
data_jsoc.insert(0, 'label', np.nan)
l = ['Bhis', 'Chis', 'Mhis', 'Xhis', 'Bhis1d', 'Chis1d', 'Mhis1d',
           'Xhis1d']
d = dict.fromkeys(l,0)
data_jsoc = data_jsoc.assign(**d)


We take the closest peak time and previous peak time, get all the values
between and classify the class according to
those times on the ```data_jsoc```. We also count the flare per AR.

In [87]:
def label_data():
    # for the current flare
    for i in range(listofresults.shape[0]-1):
        start_time = listofresults['start_time'].iloc[i]
        peak_time = listofresults['peak_time'].iloc[i]
        next_peak_time = listofresults['peak_time'].iloc[i+1]
        noaa_num = listofresults['noaa_active_region'].iloc[i]
        noaa_num_previous = noaa_num if i==0 else noaa_num_previous
        goes_class = listofresults['goes_class'].iloc[i]
        goes_flux = flare_to_flux(goes_class)
        if not noaa_num in data_jsoc['NOAA_AR'].unique():
            continue

        # if a new AR, restart count
        if noaa_num_previous != noaa_num:
            Bcount, Ccount, Mcount, Xcount = 0,0,0,0
            noaa_num_previous = noaa_num

        # get current noaa's data
        df = data_jsoc[data_jsoc['NOAA_AR'] == noaa_num]
        ar_start_time = df['T_REC'].iloc[0]

        previous_peak_time = ar_start_time if \
            listofresults['noaa_active_region'].iloc[i-1] != noaa_num else \
            listofresults['peak_time'].iloc[i-1]

        # create bolean mask and label flare class
        mask = ( (df['T_REC'] > previous_peak_time) &
                 (df['T_REC'] < peak_time))
        current_flare = df.loc[mask]
        data_jsoc.loc[data_jsoc.index[current_flare.index], 'flare'] = goes_class

        # get values after the peak
        tdelta = df['T_REC'] - peak_time
        tdelta_h = tdelta / np.timedelta64(1, "h")
        tdelta_mask = tdelta_h > 0

        # add previous flare count
        after_flare = df.loc[tdelta_mask]
        data_jsoc.loc[data_jsoc.index[after_flare.index], '{}his'.format
        (goes_class[0])] += 1

        # 1day history
        time_after = peak_time + timedelta(hours=24)
        mask = ((df['T_REC'] < time_after) &
                 (df['T_REC'] > peak_time))
        history_day_df = df.loc[mask]
        data_jsoc.loc[data_jsoc.index[history_day_df.index], '{}his1d'
            .format(goes_class[0])] += 1
        data_jsoc.loc[data_jsoc.index[history_day_df.index], 'Xmax1d'] = goes_flux


        # label positive for minimum_class_label before peak time, else label
        # negative
        if any(c in goes_class for c in minimum_class_label):
            time_before = peak_time - timedelta(hours=timedelayvariable)
            # get samples between peak and 24h before, label as positive
            mask = ((df['T_REC'] > time_before) &
                 (df['T_REC'] < peak_time))
            time_before_flare_df = df.loc[mask]
            data_jsoc.loc[data_jsoc.index[time_before_flare_df.index], 'label'] = \
                'Positive'

    # label other examples as negative
    data_jsoc['flare'] = data_jsoc['flare'].replace(np.nan, 'N')
    data_jsoc['label'] = data_jsoc['label'].replace(np.nan, 'Negative')
    return data_jsoc

# data_jsoc_labelled = label_data()

We call the labelling function.

In [88]:
# if the file isn't there, generate data
filename_data_jsoc_labelled = '../Data/SHARP/jsoc_data_labelled.csv'
if os.path.exists(filename_data_jsoc_labelled):
    data_jsoc_labelled = pd.read_csv(filename_data_jsoc_labelled)
else:
    data_jsoc_labelled = label_data()
    data_jsoc_labelled.to_csv(filename_data_jsoc_labelled, header=not os.path.exists(filename_data_jsoc_labelled),
            index=False)



# Step 4: Calculate Decay values

After the peak of a flare in an AR, the respective decay values are
calculated. with a decay rate of 12 hours.


In [89]:
def get_decay():
    data_jsoc_labelled['T_REC'] = pd.to_datetime(data_jsoc_labelled['T_REC'])
    data_jsoc_labelled['Bdec'] = 0.0
    data_jsoc_labelled['Cdec'] = 0.0
    data_jsoc_labelled['Mdec'] = 0.0
    data_jsoc_labelled['Xdec'] = 0.0
    data_jsoc_labelled['Edec'] = 0.0
    data_jsoc_labelled['logEdec'] = 0.0
    decay_tau_min = 12

    for i in range(listofresults.shape[0]-1):
        start_time = listofresults['start_time'].iloc[i]
        peak_time = listofresults['peak_time'].iloc[i]
        next_peak_time = listofresults['peak_time'].iloc[i+1]
        noaa_num = listofresults['noaa_active_region'].iloc[i]
        goes_class = listofresults['goes_class'].iloc[i]
        goescls_major = goes_class[0]
        if not noaa_num in data_jsoc['NOAA_AR'].unique():
            continue

        # get current noaa's data
        df = data_jsoc_labelled[data_jsoc['NOAA_AR'] == noaa_num]
        ar_start_time = df['T_REC'].iloc[0]

        # get values after the peak
        tdelta = df['T_REC'] - peak_time
        tdelta_h = tdelta / np.timedelta64(1, "h")
        tdelta_mask = tdelta_h < 0
        after_flare = df.loc[tdelta_mask]
        # data_jsoc_labelled.loc[data_jsoc_labelled.index[after_flare.index], '{}his'.format
        # (goes_class[0])] += 1

        # Get the decay values
        tdelta_exp = np.exp(-tdelta_h/ decay_tau_min)
        tdelta_exp[tdelta_mask] = 0.0

        df.loc[:,'{}dec'.format(goescls_major)] += tdelta_exp

        # get the energy decay values
        fl_energy_peak = 10**FL_GOESCLS_MAJOR_DICT[goescls_major] + \
                         float(goes_class[1:])

        df.loc[:,'Edec'] += tdelta_exp * fl_energy_peak

        df.loc[:,'logEdec'] += tdelta_exp * \
                                   FL_GOESCLS_MAJOR_DICT[goescls_major]

        data_jsoc_labelled.update(df)
    return data_jsoc_labelled

In [90]:
filename_data_raw = '../Data/SHARP/raw_data.csv'
if os.path.exists(filename_data_raw):
    data_raw = pd.read_csv(filename_data_raw)
else:
    data_raw = get_decay()
    data_raw.to_csv(filename_data_raw, header=not os.path.exists(filename_data_raw),
            index=False)
#todo check header



# Step 5: Normalization

Because the features have different units and scales, we
normalize the feature values as follows. For the 25 physical features, let
$z_i^k$ denote the normalized value of the ith feature of the
kth data sample. Then

\begin{align}
z_{i}^{k}=\frac{v_{i}^{k}-\mu_{i}}{\sigma_{i}}
\end{align}

where $v_i^k$ is the original value of the ith feature of the kth data
sample, $\mu _i$ is the mean of the ith feature, and $\sigma _i$ is the
standard
deviation
of the ith feature. For the 15 flare history features, we have

\begin{align}
z_{i}^{k}=\frac{v_{i}^{k}}{\max _{i}-\min _{i}}
\end{align}

where $max_i$ and $min_i$ are the maximum and minimum values of the ith
feature,
respectively.

## Cleanup data

Delete rows that have nan as in physical features.

In [91]:
sharps = ['USFLUX', 'MEANGBT', 'MEANJZH', 'MEANPOT', 'SHRGT45', 'TOTUSJH',
          'MEANGBH', 'MEANALP', 'MEANGAM', 'MEANGBZ', 'MEANJZD', 'TOTUSJZ',
          'SAVNCPP', 'TOTPOT', 'MEANSHR', 'AREA_ACR', 'R_VALUE',
          'ABSNJZH']
lorentz = ['TOTBSQ','TOTFX', 'TOTFY', 'TOTFZ', 'EPSX', 'EPSY', 'EPSZ']
data_raw = data_raw.dropna(axis=0, subset=sharps+lorentz)

Make history nans to 0

In [92]:
data_raw = data_raw.fillna(0)

Change naming scheme to match Liu

In [93]:
data_raw = data_raw.rename(columns={"T_REC": "timestep", "NOAA_AR": "NOAA",
                                  "HARPNUM":
    "HARP"})

## Splitting the data
First we need to split the data set into the training, validation and
testing sets. The normalization is fitted on the training and
validation set, and only transformed on the test set.

In [97]:
# start harp 3542
idx_val_start = data_raw[data_raw['HARP'] == 3542].index[0]
# end harp 4995
idx_val_end = data_raw[data_raw['HARP'] == 4995].index[-1]
training_set = data_raw.iloc[:idx_val_start-1,:]
validation_set = data_raw.iloc[idx_val_start:idx_val_end,:]
test_set = data_raw.iloc[idx_val_end+1:,:]


## Physical features normalization

In [98]:
startfeature = 5
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
# fit on trainval
physical_features_train = training_set.iloc[:, startfeature:startfeature+27]
physical_features_val = validation_set.iloc[:, startfeature:startfeature+27]
physical_features_test = test_set.iloc[:, startfeature:startfeature+27]
standardscaler.fit(pd.concat([physical_features_train, physical_features_val]))
new_physical_features_train = pd.DataFrame(standardscaler.transform
                                           (physical_features_train),
                                        index=physical_features_train.index,
                                        columns=physical_features_train.columns)
new_physical_features_val = pd.DataFrame(standardscaler.transform
                                           (physical_features_val),
                                        index=physical_features_val.index,
                                        columns=physical_features_val.columns)
new_physical_features_test = pd.DataFrame(standardscaler.transform
                                          (physical_features_test),
                                        index=physical_features_test.index,
                                        columns=physical_features_test.columns)

## History normalization

In [99]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
his_features_train = training_set.iloc[:, startfeature+28:]
his_features_val = validation_set.iloc[:, startfeature+28:]
his_features_test = test_set.iloc[:, startfeature+28:]
minmaxscaler.fit(pd.concat([his_features_train, his_features_val]))
new_his_features_train = pd.DataFrame(minmaxscaler.transform
                                           (his_features_train),
                                        index=his_features_train.index,
                                        columns=his_features_train.columns)
new_his_features_val = pd.DataFrame(minmaxscaler.transform
                                           (his_features_val),
                                        index=his_features_val.index,
                                        columns=his_features_val.columns)
new_his_features_test = pd.DataFrame(minmaxscaler.transform
                                          (his_features_test),
                                        index=his_features_test.index,
                                        columns=his_features_test.columns)

In [100]:
# make new normalized dataset
normalized_train = pd.concat([data_raw.iloc[:idx_val_start-1,:startfeature],
                             new_physical_features_train,
                              new_his_features_train],
                            axis=1)
normalized_val = pd.concat([data_raw.iloc[idx_val_start:idx_val_end,:startfeature],
                             new_physical_features_val, new_his_features_val],
                            axis=1)
normalized_test = pd.concat([data_raw.iloc[idx_val_end+1:,:startfeature],
                             new_physical_features_test,
                             new_his_features_test],
                            axis=1)



Save to csv

In [101]:
filename_krynauw = '../Data/Krynauw/'
training_set.to_csv(filename_krynauw + 'training.csv', index=False)
validation_set.to_csv(filename_krynauw + 'validation.csv', index=False)
test_set.to_csv(filename_krynauw + 'testing.csv', index=False)

normalized_train.to_csv(filename_krynauw + 'normalized_training.csv', index=False)
normalized_val.to_csv(filename_krynauw + 'normalized_validation.csv', index=False)
normalized_test.to_csv(filename_krynauw + 'normalized_testing.csv', index=False)
